In [1]:
import os
import time
import random
import re
import pandas as pd
import numpy as np
import mysql.connector

# 读取数据

In [19]:
news_df = pd.read_csv('../data/news_data/event.csv')

In [20]:
news_df = news_df.fillna('')

# 删除原有数据库

In [18]:
cnx = mysql.connector.connect(user='root', password='ir@DUT#2018',
                              host='10.8.128.205', port='23306')

cursor = cnx.cursor()

In [ ]:
cursor.execute('drop database newsDB;')
cnx.close()

# 新建数据库 符合utf8mb4格式数据

In [2]:
cnx = mysql.connector.connect(user='root', password='ir@DUT#2018',
                              host='10.8.128.205', port='23306')

cursor = cnx.cursor()

In [6]:
cursor.execute('create database newsDB character set UTF8mb4 collate utf8mb4_bin;')

In [7]:
cnx.close()

# 创建数据表

In [13]:
cnx = mysql.connector.connect(user='news_user', password='ir@DUT#news',
                              host='10.8.128.205',database='newsDB', port='23306')

cursor = cnx.cursor()

In [11]:
create_sql = '''
CREATE TABLE IF NOT EXISTS `event` (
  `GLOBALEVENTID` varchar(255) DEFAULT NULL,
  `SQLDATE` varchar(255) DEFAULT NULL,
  `MonthYear` varchar(255) DEFAULT NULL,
  `Year` varchar(255) DEFAULT NULL,
  `FractionDate` varchar(255) DEFAULT NULL,
  `Actor1Code` varchar(255) DEFAULT NULL,
  `Actor1Name` varchar(255) DEFAULT NULL,
  `Actor1CountryCode` varchar(255) DEFAULT NULL,
  `Actor1KnownGroupCode` varchar(255) DEFAULT NULL,
  `Actor1EthnicCode` varchar(255) DEFAULT NULL,
  `Actor1Religion1Code` varchar(255) DEFAULT NULL,
  `Actor1Religion2Code` varchar(255) DEFAULT NULL,
  `Actor1Type1Code` varchar(255) DEFAULT NULL,
  `Actor1Type2Code` varchar(255) DEFAULT NULL,
  `Actor1Type3Code` varchar(255) DEFAULT NULL,
  `Actor2Code` varchar(255) DEFAULT NULL,
  `Actor2Name` varchar(255) DEFAULT NULL,
  `Actor2CountryCode` varchar(255) DEFAULT NULL,
  `Actor2KnownGroupCode` varchar(255) DEFAULT NULL,
  `Actor2EthnicCode` varchar(255) DEFAULT NULL,
  `Actor2Religion1Code` varchar(255) DEFAULT NULL,
  `Actor2Religion2Code` varchar(255) DEFAULT NULL,
  `Actor2Type1Code` varchar(255) DEFAULT NULL,
  `Actor2Type2Code` varchar(255) DEFAULT NULL,
  `Actor2Type3Code` varchar(255) DEFAULT NULL,
  `IsRootEvent` varchar(255) DEFAULT NULL,
  `EventCode` varchar(255) DEFAULT NULL,
  `EventBaseCode` varchar(255) DEFAULT NULL,
  `EventRootCode` varchar(255) DEFAULT NULL,
  `QuadClass` varchar(255) DEFAULT NULL,
  `GoldsteinScale` varchar(255) DEFAULT NULL,
  `NumMentions` varchar(255) DEFAULT NULL,
  `NumSources` varchar(255) DEFAULT NULL,
  `NumArticles` varchar(255) DEFAULT NULL,
  `AvgTone` varchar(255) DEFAULT NULL,
  `Actor1Geo_Type` varchar(255) DEFAULT NULL,
  `Actor1Geo_FullName` varchar(255) DEFAULT NULL,
  `Actor1Geo_CountryCode` varchar(255) DEFAULT NULL,
  `Actor1Geo_ADM1Code` varchar(255) DEFAULT NULL,
  `Actor1Geo_ADM2Code` varchar(255) DEFAULT NULL,
  `Actor1Geo_Lat` varchar(255) DEFAULT NULL,
  `Actor1Geo_Long` varchar(255) DEFAULT NULL,
  `Actor1Geo_FeatureID` varchar(255) DEFAULT NULL,
  `Actor2Geo_Type` varchar(255) DEFAULT NULL,
  `Actor2Geo_FullName` varchar(255) DEFAULT NULL,
  `Actor2Geo_CountryCode` varchar(255) DEFAULT NULL,
  `Actor2Geo_ADM1Code` varchar(255) DEFAULT NULL,
  `Actor2Geo_ADM2Code` varchar(255) DEFAULT NULL,
  `Actor2Geo_Lat` varchar(255) DEFAULT NULL,
  `Actor2Geo_Long` varchar(255) DEFAULT NULL,
  `Actor2Geo_FeatureID` varchar(255) DEFAULT NULL,
  `ActionGeo_Type` varchar(255) DEFAULT NULL,
  `ActionGeo_FullName` varchar(255) DEFAULT NULL,
  `ActionGeo_CountryCode` varchar(255) DEFAULT NULL,
  `ActionGeo_ADM1Code` varchar(255) DEFAULT NULL,
  `ActionGeo_ADM2Code` varchar(255) DEFAULT NULL,
  `ActionGeo_Lat` varchar(255) DEFAULT NULL,
  `ActionGeo_Long` varchar(255) DEFAULT NULL,
  `ActionGeo_FeatureID` varchar(255) DEFAULT NULL,
  `DATEADDED` varchar(255) DEFAULT NULL,
  `SOURCEURL` varchar(255) DEFAULT NULL,
  `MediaGeo` varchar(255) DEFAULT NULL,
  `Title` text,
  `Content` mediumtext
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_bin;
'''

In [12]:
create_sql = create_sql.replace('\n','')

In [14]:
cursor.execute(create_sql)

## 插入数据样例代码

In [15]:
cnx = mysql.connector.connect(user='news_user', password='ir@DUT#news',
                              host='10.8.128.205',database='newsDB', port='23306')

cursor = cnx.cursor()

In [16]:
head_sql = 'INSERT INTO `event` (`GLOBALEVENTID`, `SQLDATE`, `MonthYear`, `Year`, `FractionDate`, `Actor1Code`, `Actor1Name`, `Actor1CountryCode`, `Actor1KnownGroupCode`, `Actor1EthnicCode`, `Actor1Religion1Code`, `Actor1Religion2Code`, `Actor1Type1Code`, `Actor1Type2Code`, `Actor1Type3Code`, `Actor2Code`, `Actor2Name`, `Actor2CountryCode`, `Actor2KnownGroupCode`, `Actor2EthnicCode`, `Actor2Religion1Code`, `Actor2Religion2Code`, `Actor2Type1Code`, `Actor2Type2Code`, `Actor2Type3Code`, `IsRootEvent`, `EventCode`, `EventBaseCode`, `EventRootCode`, `QuadClass`, `GoldsteinScale`, `NumMentions`, `NumSources`, `NumArticles`, `AvgTone`, `Actor1Geo_Type`, `Actor1Geo_FullName`, `Actor1Geo_CountryCode`, `Actor1Geo_ADM1Code`, `Actor1Geo_ADM2Code`, `Actor1Geo_Lat`, `Actor1Geo_Long`, `Actor1Geo_FeatureID`, `Actor2Geo_Type`, `Actor2Geo_FullName`, `Actor2Geo_CountryCode`, `Actor2Geo_ADM1Code`, `Actor2Geo_ADM2Code`, `Actor2Geo_Lat`, `Actor2Geo_Long`, `Actor2Geo_FeatureID`, `ActionGeo_Type`, `ActionGeo_FullName`, `ActionGeo_CountryCode`, `ActionGeo_ADM1Code`, `ActionGeo_ADM2Code`, `ActionGeo_Lat`, `ActionGeo_Long`, `ActionGeo_FeatureID`, `DATEADDED`, `SOURCEURL`, `MediaGeo`, `Title`, `Content`) VALUES'

In [17]:
startpoint = 0

# 目前失败 没法子入库

In [21]:
for num,i in enumerate(news_df.itertuples()):
    if num < startpoint:
        continue
    if num%1000==0:
        print('[{0}] Num:{1}'.format(time.ctime(),num))
    sql_data = [str(p).replace('\'','') for p in i[1:]]
    val_str = ["'{0}'".format(p) for p in sql_data]
    tail_sql = '({0})'.format(','.join(val_str))
    sql_str = head_sql+tail_sql
    sql_str = sql_str.replace('\\','')
    cursor.execute(sql_str)

cnx.commit()

cnx.close()

[Tue Jul 23 00:09:16 2019] Num:0
[Tue Jul 23 00:09:19 2019] Num:1000


DatabaseError: 1366 (HY000): Incorrect string value: '\xF0\x9F\x87\xB1\xF0\x9F...' for column 'Content' at row 1

In [17]:
num

1212

In [23]:
cnx.close()

In [17]:
cursor.execute(detail_sql_create_str)

In [18]:
cnx.close()